# 구내식당 식수인원예측
Predict_Food_Demand_Model

- 중식계, 석식계 따로 예측

# Settings

In [311]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc

import warnings
import numpy as np
import pandas as pd

import seaborn as sns
plt.style.use('seaborn')
sns.set(font_scale=1.5) 

# warning 출력 ignore
warnings.filterwarnings(action='ignore')

# 그래프에서 '-' 기호 때문에 문제 발생하는것 방지
mpl.rcParams['axes.unicode_minus'] = False

# 한글폰트 적용
font_path = 'C:/jupyter_home/font/malgun.ttf'
font_name = fm.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [312]:
import re    # 정규식
import itertools

In [313]:
from sklearn import linear_model
from scipy import stats            # 이상치처리
from sklearn.preprocessing import MinMaxScaler

# Import

In [315]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

- 일자
- 요일
- 본사정원수 -
- 본사휴가자수 ★
- 본사출장자수 ★
- 본사시간외근무명령서승인건수 ★
- 현본사소속재택근무자수 -
- 조식메뉴
- 중식메뉴
- 석식메뉴
- 중식계 predict
- 석식계 predict


- 중식메뉴, 석식메뉴를 따로 처리해서 따로 할 수 있도록...

In [316]:
df_train.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0


In [317]:
df_test.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,2021-01-27,수,2983,88,182,5,358.0,모닝롤/연유버터베이글 우유/주스 계란후라이/찐계란 단호박죽/흑미밥 우거지국 고기완자...,쌀밥/흑미밥/찰현미밥 대구지리 매운돈갈비찜 오꼬노미계란말이 상추무침 포기김치 양상추...,흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치
1,2021-01-28,목,2983,104,212,409,348.0,모닝롤/대만샌드위치 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 황태국 시래기지짐 ...,쌀밥/보리밥/찰현미밥 우렁된장찌개 오리주물럭 청양부추전 수제삼색무쌈 겉절이김치 양상...,충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지
2,2021-01-29,금,2983,270,249,0,294.0,모닝롤/핫케익 우유/주스 계란후라이/찐계란 오곡죽/흑미밥 매생이굴국 고구마순볶음 양...,쌀밥/흑미밥/찰현미밥 팽이장국 수제돈까스*소스 가자미조림 동초나물무침 포기김치 양상...,흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치


# Pre Processing : df_train_1, df_test_1
- 본사 휴가자수, 본사출장자수, 본사시간외근무명령서승인건수, 현본사소속재택근무자수 에 대해서 

In [326]:
# pre processing 필요없는 컬럼 삭제

df_train_1 = df_train.drop(['요일', '본사정원수', '조식메뉴','중식메뉴','석식메뉴'], axis=1)
df_train_1.rename(columns = {'본사휴가자수' : '휴가자수',
                               '본사출장자수' : '출장자수',
                               '본사시간외근무명령서승인건수': '승인건수',
                               '현본사소속재택근무자수': '재택근무'}, inplace = True)
df_test_1 = df_test.drop(['요일', '본사정원수', '조식메뉴','중식메뉴','석식메뉴'], axis=1)
df_test_1.rename(columns = {'본사휴가자수' : '휴가자수',
                               '본사출장자수' : '출장자수',
                               '본사시간외근무명령서승인건수': '승인건수',
                               '현본사소속재택근무자수': '재택근무'}, inplace = True)


## 1. 결측치 확인

In [327]:
df_train_1.isnull().sum()

일자      0
휴가자수    0
출장자수    0
승인건수    0
재택근무    0
중식계     0
석식계     0
dtype: int64

In [328]:
df_test_1.isnull().sum()

일자      0
휴가자수    0
출장자수    0
승인건수    0
재택근무    0
dtype: int64

## 2. 이상치 확인
- 이상치가 아닌 실제 데이터라고 생각하고 이상치 제거를 하지 않겠음

## 3. 정규화

- 전체에 대해 Min-Max 정규화를 수행
- 몇개 column[\'본사휴가자수\', \'본사출장자수\']에 log 함수를 씌워서 정규분포로 만들어준다

### A) log 사용해서 정규분포화

In [321]:
# df_train_1['휴가자수'] = pd.Series(stats.boxcox(df_train_1['휴가자수']+0.0001)[0])
# df_train_1['출장자수'] = pd.Series(stats.boxcox(df_train_1['출장자수']+0.0001)[0])

# df_test_1['휴가자수'] = pd.Series(stats.boxcox(df_test_1['휴가자수']+0.0001)[0])
# df_test_1['출장자수'] = pd.Series(stats.boxcox(df_test_1['출장자수']+0.0001)[0])

In [329]:
df_train_1['휴가자수'] = np.log1p(df_train_1['휴가자수'])
df_train_1['출장자수'] = np.log1p(df_train_1['출장자수'])

df_test_1['휴가자수'] = np.log1p(df_test_1['휴가자수'])
df_test_1['출장자수'] = np.log1p(df_test_1['출장자수'])

In [330]:
df_train_1.head()

,일자,휴가자수,출장자수,승인건수,재택근무,중식계,석식계
0,2016-02-01,3.931826,5.017280,238,0.0,1039.0,331.0
1,2016-02-02,3.931826,5.159055,319,0.0,867.0,560.0
2,2016-02-03,4.043051,5.198497,111,0.0,1017.0,573.0
3,2016-02-04,4.653960,5.398163,355,0.0,978.0,525.0
4,2016-02-05,5.631212,5.204007,34,0.0,925.0,330.0


In [331]:
df_test_1.head()

,일자,휴가자수,출장자수,승인건수,재택근무
0,2021-01-27,4.488636,5.209486,5,358.0
1,2021-01-28,4.653960,5.361292,409,348.0
2,2021-01-29,5.602119,5.521461,0,294.0
3,2021-02-01,4.691348,5.043425,538,322.0
4,2021-02-02,4.143135,5.231109,455,314.0


### B) Min-Max 정규화

- train, test data 모두 MinMax 정규화를 해줄것이다.
- scaler_x, scaler_t1, scaler_t2 scaler를 만들어준다.

In [332]:
scaler = MinMaxScaler()

df_train_2 = df_train_1.copy()
df_test_2 = df_test_1.copy()

df_train_2.iloc[:,1:] = scaler.fit(df_train_1.iloc[:,1:]).transform(df_train_1.iloc[:,1:])
df_test_2.iloc[:,1:] = scaler.fit(df_train_1.iloc[:,1:-2]).transform(df_test_1.iloc[:,1:])

df_train_2 = pd.DataFrame(df_train_2)
df_train_2.columns = ['일자', '휴가자수', '출장자수', '승인건수', '재택근무', '중식계', '석식계']

df_test_2 = pd.DataFrame(df_test_2)
df_test_2.columns = ['일자', '휴가자수', '출장자수', '승인건수', '재택근무']

In [333]:
df_train_2.head()

,일자,휴가자수,출장자수,승인건수,재택근무,중식계,석식계
0,2016-02-01,0.191671,0.581676,0.227969,0.0,0.638865,0.365746
1,2016-02-02,0.191671,0.646124,0.305556,0.0,0.490972,0.618785
2,2016-02-03,0.219953,0.664053,0.106322,0.0,0.619948,0.633149
3,2016-02-04,0.375296,0.754815,0.340038,0.0,0.586414,0.580110
4,2016-02-05,0.623794,0.666557,0.032567,0.0,0.540843,0.364641


In [334]:
df_test_2.head()

,일자,휴가자수,출장자수,승인건수,재택근무
0,2021-01-27,0.333258,0.669048,0.004789,0.671670
1,2021-01-28,0.375296,0.738055,0.391762,0.652908
2,2021-01-29,0.616396,0.810863,0.000000,0.551595
3,2021-02-01,0.384803,0.593561,0.515326,0.604128
4,2021-02-02,0.245403,0.678877,0.435824,0.589118


## 4. 메뉴 one-hot encoding
- 확인해본결과 상관성이 전혀없어서 넣지 않기로함

# Model

<선형 회귀 모델 종류>
- 일반선형회귀모델
- 릿지회귀모델
- 라쏘회귀모델
- 엘라스틱넷 회귀모델
- 로지스틱 회귀모델

# 중식계

In [337]:
df_train_lunch = df_train_2.drop(['일자','석식계'], axis=1)
df_test_lunch = df_test_2

In [338]:
train, val = train_test_split(df_train_lunch, test_size=0.3,random_state=0)
X_train = train[train.columns[:-1]]
y_train = train[train.columns[-1]]
X_val = val[val.columns[:-1]]
y_val = val[val.columns[-1]]
X_test= df_test_lunch

### Learning

#### sklearn leaner_model : 중식계

In [340]:
# sklearn 구현
from sklearn import linear_model

model = linear_model.LinearRegression()

model.fit(X_train, y_train)

prediction1 = model.predict(X_val)

# print('Accuracy for rbf sklearn leaner_model is ',metrics.precision_score(prediction1,y_val))
# 정확도 : 전체 샘플 중 맞게 예측한 샘플 수의 비율

print(f'MAE : {np.mean(y_val - prediction1)}')
print(f'MSE : {np.mean(np.power(y_val - prediction1, 2))}')
print(f'RMSE : {np.sqrt(np.mean(np.power(y_val - prediction1, 2)))}')


MAE : -0.0023243132732201337
MSE : 0.017007429910053995
RMSE : 0.1304125373959651


### Prediction

#### sklearn leaner_model : 중식계

In [344]:
model = linear_model.LinearRegression()

model.fit(X_train, y_train)

prediction1 = model.predict(df_test_lunch.iloc[:,1:])

scaler_t1 = scaler.fit(df_train_1['중식계'].values.reshape(-1,1))

prediction2 = scaler_t1.inverse_transform(prediction1.reshape(-1,1))

df_result_lunch = df_test_lunch.copy()
df_result_lunch['중식계'] = pd.DataFrame(prediction1)

# display(df_result_lunch.head())

df_result_lunch['중식계'] = pd.DataFrame(prediction2)

display(df_result_lunch.head())

,일자,휴가자수,출장자수,승인건수,재택근무,중식계
0,2021-01-27,0.333258,0.669048,0.004789,0.671670,0.532806
1,2021-01-28,0.375296,0.738055,0.391762,0.652908,0.549253
2,2021-01-29,0.616396,0.810863,0.000000,0.551595,0.287120
3,2021-02-01,0.384803,0.593561,0.515326,0.604128,0.719405
4,2021-02-02,0.245403,0.678877,0.435824,0.589118,0.679253


,일자,휴가자수,출장자수,승인건수,재택근무,중식계
0,2021-01-27,0.333258,0.669048,0.004789,0.671670,915.653330
1,2021-01-28,0.375296,0.738055,0.391762,0.652908,934.781017
2,2021-01-29,0.616396,0.810863,0.000000,0.551595,629.921067
3,2021-02-01,0.384803,0.593561,0.515326,0.604128,1132.667754
4,2021-02-02,0.245403,0.678877,0.435824,0.589118,1085.971560


# 석식계

In [345]:
df_train_dinner = df_train_2.drop(['일자','중식계'], axis=1)
df_test_dinner = df_test_2

In [346]:
train, val = train_test_split(df_train_dinner, test_size=0.3,random_state=0)
X_train = train[train.columns[:-1]]
y_train = train[train.columns[-1]]
X_val = val[val.columns[:-1]]
y_val = val[val.columns[-1]]
X_test= df_test_dinner

### Learning

#### sklearn leaner_model : 석식계

In [349]:
# sklearn 구현
from sklearn import linear_model

model = linear_model.LinearRegression()

model.fit(X_train, y_train)

prediction1 = model.predict(X_val)

# print('Accuracy for rbf sklearn leaner_model is ',metrics.precision_score(prediction1,y_val))
# 정확도 : 전체 샘플 중 맞게 예측한 샘플 수의 비율

print(f'MAE : {np.mean(y_val - prediction1)}')
print(f'MSE : {np.mean(np.power(y_val - prediction1, 2))}')
print(f'RMSE : {np.sqrt(np.mean(np.power(y_val - prediction1, 2)))}')


MAE : -0.005002532881634871
MSE : 0.013263570188779092
RMSE : 0.11516757438089548


### Prediction

#### sklearn leaner_model : 석식계

In [354]:
model = linear_model.LinearRegression()

model.fit(X_train, y_train)

prediction1 = model.predict(df_test_dinner.iloc[:,1:])

scaler_t1 = scaler.fit(df_train_1['석식계'].values.reshape(-1,1))

prediction2 = scaler_t1.inverse_transform(prediction1.reshape(-1,1))

df_result_dinner = df_test_dinner.copy()
df_result_dinner['석식계'] = pd.DataFrame(prediction1)

display(df_result_dinner.head())

df_result_dinner['석식계'] = pd.DataFrame(prediction2)

display(df_result_dinner.head())

,일자,휴가자수,출장자수,승인건수,재택근무,석식계
0,2021-01-27,0.333258,0.669048,0.004789,0.671670,0.388060
1,2021-01-28,0.375296,0.738055,0.391762,0.652908,0.497755
2,2021-01-29,0.616396,0.810863,0.000000,0.551595,0.321089
3,2021-02-01,0.384803,0.593561,0.515326,0.604128,0.576103
4,2021-02-02,0.245403,0.678877,0.435824,0.589118,0.556859


,일자,휴가자수,출장자수,승인건수,재택근무,석식계
0,2021-01-27,0.333258,0.669048,0.004789,0.671670,351.194416
1,2021-01-28,0.375296,0.738055,0.391762,0.652908,450.468446
2,2021-01-29,0.616396,0.810863,0.000000,0.551595,290.585132
3,2021-02-01,0.384803,0.593561,0.515326,0.604128,521.373237
4,2021-02-02,0.245403,0.678877,0.435824,0.589118,503.957626


# 데이터 합치기

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv')
submission.head()

In [355]:
df_result_lunch.head()

,일자,휴가자수,출장자수,승인건수,재택근무,중식계
0,2021-01-27,0.333258,0.669048,0.004789,0.671670,915.653330
1,2021-01-28,0.375296,0.738055,0.391762,0.652908,934.781017
2,2021-01-29,0.616396,0.810863,0.000000,0.551595,629.921067
3,2021-02-01,0.384803,0.593561,0.515326,0.604128,1132.667754
4,2021-02-02,0.245403,0.678877,0.435824,0.589118,1085.971560


In [356]:
df_result_dinner.head()

,일자,휴가자수,출장자수,승인건수,재택근무,석식계
0,2021-01-27,0.333258,0.669048,0.004789,0.671670,351.194416
1,2021-01-28,0.375296,0.738055,0.391762,0.652908,450.468446
2,2021-01-29,0.616396,0.810863,0.000000,0.551595,290.585132
3,2021-02-01,0.384803,0.593561,0.515326,0.604128,521.373237
4,2021-02-02,0.245403,0.678877,0.435824,0.589118,503.957626


In [359]:
submission['중식계'] = df_result_lunch['중식계']
submission['석식계'] = df_result_dinner['석식계']

submission.head()

,일자,중식계,석식계
0,2021-01-27,915.653330,351.194416
1,2021-01-28,934.781017,450.468446
2,2021-01-29,629.921067,290.585132
3,2021-02-01,1132.667754,521.373237
4,2021-02-02,1085.971560,503.957626


In [360]:
submission.to_csv('./data/submission_first.csv', index=False)

- 출장자수, 등 너무차이나는거는 category로 해볼까?
